In [2]:
%pylab notebook
%load_ext autoreload
%autoreload 2

#from mpl_toolkits.mplot3d import Axes3D
import scipy.constants as cst
from lammpsrun import LAMMPS, Prism
from itertools import product

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.neighborlist import neighbor_list
from ase.build import *
from ase.md import *
from ase.spacegroup import crystal

homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
print(homepath)

Populating the interactive namespace from numpy and matplotlib
D:\Cloud Storage\GitRepositories\LAMMPS_Simulation\


Crystallographic data is obtained from https://materials.springer.com/isp/crystallographic/docs/sd_1628167

# Bulk siderite crystal

In [3]:
siderite = crystal(symbols = ['Fe', 'C', 'O'],
                    basis = [(0, 0, 0), 
                             (0, 0, 0.25),
                             (0.2748, 0, 0.25)],
                    spacegroup=167,
                    cellpar=[4.72, 4.72, 15.51, 90, 90, 120])

# Increasing size to improve QEq accuracy (>1 nm)
siderite_bulk = siderite.repeat((3, 3, 1))
siderite_bulk.center()

In [5]:
# Atoms C Fe O
siderite_bulk_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 12.011",
             "2 55.845",
             "3 15.999"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' C Fe O'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax iso 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

In [20]:
# Setting up simulation input
siderite_bulk_calc = LAMMPS(tmp_dir = homepath + "temp", 
                            parameters = siderite_bulk_reaxff_params,
                            no_data_file=True)

siderite_bulk.set_calculator(siderite_bulk_calc)
siderite_bulk_calc.atoms = siderite_bulk.copy()
siderite_bulk_calc.prism = Prism(siderite_bulk_calc.atoms.get_cell())

In [21]:
# Writing input file
siderite_bulk_calc.write_lammps_in(lammps_in = homepath + "siderite\\siderite_bulk_reaxff_minimize.in",
                                   lammps_trj = homepath + "siderite\\siderite_bulk_reaxff_minimize.lammpstrj")


In [ ]:
# Running calculations
!lmp_serial -in siderite\siderite_bulk_reaxff_minimize.in -log siderite\siderite_bulk_reaxff_minimize.log

In [23]:
# Reading LAMMPS output
siderite_bulk_calc.read_lammps_log(lammps_log = homepath + "siderite\\siderite_bulk_reaxff_minimize.log")
siderite_bulk_data = siderite_bulk_calc.thermo_content[-1]

siderite_bulk_calc.trajectory_out = Trajectory(homepath + "siderite\\siderite_bulk_reaxff_minimize.traj", 'w')
siderite_bulk_calc.read_lammps_trj(lammps_trj = homepath + "siderite\\siderite_bulk_reaxff_minimize.lammpstrj")
siderite_bulk_calc.trajectory_out.close()

In [24]:
siderite_bulk_traj = Trajectory(homepath + "siderite\\siderite_bulk_reaxff_minimize.traj", 'r')

siderite_bulk_traj_view = view(siderite_bulk_traj, viewer='ngl')
siderite_bulk_traj_view.view.update_spacefill(radiusType='covalent',
                                              scale=0.8,
                                              color_scheme='element',
                                              color_scale='rainbow')
siderite_bulk_traj_view.view.player.step = 1
siderite_bulk_traj_view

# Surfaces/Slabs

Past work done with surface 104, 110, 100, 101, 001 and 012. High surface energy on 100 and 101

In [6]:
# Atoms C Fe O
siderite_slab_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 12.011",
             "2 55.845",
             "3 15.999"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' C Fe O'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax xy 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

## siderite (104)

In [6]:
siderite_surfaces = ['104', '110', '100', '101', '001', '012']
for s_surface in siderite_surfaces:
    sCurrentSurface = surface(siderite, (int(s_surface[0]), int(s_surface[1]), int(s_surface[2])), 2, vacuum=10)
    sCurrentSurface = sCurrentSurface.repeat((2, 2, 1))
    view(sCurrentSurface)